In [1]:
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.cm as cmx

In [2]:
clusters = pd.read_json("./files/cluster.json")

ValueError: Expected object or value

In [3]:
def plotSteps(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        iteration = data.iloc[runIdx]["iteration"]
        clusters = data.iloc[runIdx]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters))
        print("iteration: %s, clusters: %s" % (iteration, len(clusters)))
        for cIdx in range(0, len(clusters)):
            rgba_color = cm.gnuplot(norm(cIdx))
            pointsX = [point[0] for point in clusters[cIdx]["points"]]
            pointsY = [point[1] for point in clusters[cIdx]["points"]]
            ax.scatter(pointsX, pointsY, color=rgba_color)
            ax.scatter(clusters[cIdx]["centroid"][0], clusters[cIdx]["centroid"][1], color=rgba_color, marker="+")

        plt.show()

In [4]:
plotSteps(clusters)

NameError: name 'clusters' is not defined

In [5]:
def plot(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        clusters_step1 = data.iloc[runIdx]["steps"][0]["clusters"]
        clusters_step2 = data.iloc[runIdx]["steps"][1]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters_step1))
        for cIdx in range(0, len(clusters_step1)):
            rgba_color = cm.gnuplot(norm(cIdx))
            ax.plot(clusters_step1[cIdx]["syntheticCenter"], "--", color=rgba_color, label="cluster %s" % cIdx)
            ax.plot(clusters_step2[cIdx]["syntheticCenter"], color=rgba_color, label="cluster %s - rescheduled" % cIdx)
            print(clusters_step2[cIdx]["metric"])
        ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)

        ax.set(xlabel='Hours', ylabel='Power (kW)',title='')
        plt.show()

In [6]:
def plot_aggm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. agg m"], "--", color=rgba_color, label="s1. agg m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. agg m"], color=rgba_color, label="s2. agg m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_maxm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. max m"], "--", color=rgba_color, label="s1. max m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. max m"], color=rgba_color, label="s2. max m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Max. Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_peak(summary, file_name=None):
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. peak"], "--", color=rgba_color, label="s1. peak")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. peak"], color=rgba_color, label="s2. peak")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Peak power (kW)',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig

# PAR Aggregate

In [54]:
summaryBatch_global = pd.read_json("./files/summaryBatch.json")
#summaryBatch_global = summaryBatch_global[["k","clusters","s1. agg m","s1. max m","s1. peak","s2. agg m","s2. max m","s2. peak","total m"]]
#batch_global = pd.read_json("./files/batch.json")

index_list = []
step_list = []
k = len(json_normalize(summaryBatch_global.iloc[0]["step"]))

for i in range(0 , len(summaryBatch_global)):
    step = json_normalize(summaryBatch_global.iloc[i]["step"])
    crossfold = json_normalize(summaryBatch_global.iloc[i]["crossfold"])
    for j in range(0, k):
        index_list.append((crossfold["splits"].values[0], float("{0:.2f}".format(crossfold["sampleSize"].values[0])), step["k"].values[j]))
        step_list.append(step.iloc[j])

    
index = pd.MultiIndex.from_tuples(index_list, names=["split", "sampleSize", "k"])
summaryBatch_global_treated = pd.DataFrame(step_list, index=index)
summaryBatch_global_treated

clusters  k  s1. agg m  s1. max m  \
split sampleSize k                                                      
1     0.6        1                     [113]  1   1.381087   1.381087   
                 2                  [57, 56]  2   1.001771   1.450742   
                 3              [37, 37, 39]  3   1.049068   1.483215   
                 4          [28, 28, 28, 29]  4   1.015177   1.543310   
                 5      [23, 23, 23, 22, 22]  5   1.046066   1.595152   
                 6  [18, 18, 19, 20, 18, 20]  6   1.014670   1.529094   
      0.7        1                     [132]  1   1.369004   1.369004   
                 2                  [66, 66]  2   1.008400   1.388403   
                 3              [44, 44, 44]  3   1.011260   1.434968   
                 4          [33, 33, 33, 33]  4   1.010871   1.447178   
                 5      [26, 27, 27, 26, 26]  5   1.019723   1.485373   
                 6  [22, 22, 22, 22, 22, 22]  6   1.015998   1.532538   
      0.8        1                     [151]  1   1.430176   1.430176   
                 2                  [75, 76]  2   1.000305   1.430600   
                 3              [51, 50, 50]  3   1.000900   1.460055   
                 4          [37, 37, 38, 39]  4   1.014247   1.490767   
                 5      [30, 30, 31, 30, 30]  5   1.038327   1.521564   
                 6  [25, 25, 25, 26, 25, 25]  6   1.033670   1.577348   
      0.9        1                     [170]  1   1.357453   1.357453   
                 2                  [85, 85]  2   1.001074   1.381724   
                 3              [58, 56, 56]  3   1.014611   1.386491   
                 4          [42, 42, 42, 44]  4   1.008931   1.414701   
                 5      [34, 34, 34, 34, 34]  5   1.010901   1.437743   
                 6  [28, 28, 28, 29, 28, 29]  6   1.044671   1.500470   
      1.0        1                     [189]  1   1.371114   1.371114   
                 2                  [95, 94]  2   1.000200   1.371350   
                 3              [63, 63, 63]  3   1.014644   1.395779   
                 4          [47, 47, 47, 48]  4   1.064519   1.475421   
                 5      [37, 37, 37, 38, 40]  5   1.021058   1.472670   
                 6  [31, 31, 31, 31, 31, 34]  6   1.051584   1.481487   

                      s1. peak   total m  
split sampleSize k                        
1     0.6        1  386.778517  1.381087  
                 2  203.156483  1.381087  
                 3  228.332900  1.381087  
                 4  124.446267  1.381087  
                 5  105.367167  1.381087  
                 6   80.292967  1.381087  
      0.7        1  454.623917  1.369004  
                 2  249.367883  1.369004  
                 3  203.604717  1.369004  
                 4  139.163350  1.369004  
                 5  126.703817  1.369004  
                 6  141.064933  1.369004  
      0.8        1  574.109500  1.430176  
                 2  295.001350  1.430176  
                 3  202.867983  1.430176  
                 4  243.858583  1.430176  
                 5  135.716650  1.430176  
                 6  133.402833  1.430176  
      0.9        1  607.573067  1.357453  
                 2  314.625167  1.357453  
                 3  215.214050  1.357453  
                 4  168.857500  1.357453  
                 5  210.368683  1.357453  
                 6  136.814033  1.357453  
      1.0        1  676.160150  1.371114  
                 2  385.780117  1.371114  
                 3  258.705150  1.371114  
                 4  352.525767  1.371114  
                 5  244.149433  1.371114  
                 6  222.809117  1.371114

SyntaxError: invalid syntax (<ipython-input-58-132c0c9d5401>, line 1)

In [67]:
k2 = 2
query = f"k == {k2}"
summaryBatch_global_treated.query(query)["s1. agg m"].mean()
summaryBatch_global_treated.query(query)["s1. max m"].mean()
summaryBatch_global_treated.query(query)["s1. peak"].mean()

289.58619999999996

In [ ]:
summaryBatch_global

In [9]:
with open('summaryBatch.tex','w') as tf:
    tf.write(summaryBatch_global.to_latex())

In [52]:
x = 3.24234
float("{0:.2f}".format(x))

3.24